## make adjacency matrix to export

In [62]:
from pathlib import Path
import os
import numpy as np
import networkx as nx

btw, pathlib.Path is amazing, just started playing with it, fully object-oriented path manipulation

### set up paths, etc

In [63]:
p = Path('DK_res-1x1x1')

In [64]:
# making sure I know what iterdir() does
[i for i in p.iterdir()][:2]

[PosixPath('DK_res-1x1x1/sub-NDARFR109LKT_acq-64dir_dwi_DK_res-1x1x1_measure-spatial-ds._elist.csv'),
 PosixPath('DK_res-1x1x1/sub-NDARCT933HF2_acq-64dir_dwi_DK_res-1x1x1_measure-spatial-ds._elist.csv')]

### grab graphs

In [65]:
# list of numpy matrices
graphs = [nx.read_weighted_edgelist(i, delimiter=',') 
          for i in p.iterdir()]

### check nodelists, output sorted version to numpy matrix

In [66]:
graphs[0].nodes()

NodeView(('1', '2', '3', '4', '5', '6', '7', '8', '10', '12', '16', '17', '22', '23', '24', '25', '28', '30', '32', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '48', '49', '53', '77', '78', '80', '83', '18', '19', '20', '26', '64', '66', '71', '73', '76', '79', '9', '29', '31', '50', '51', '58', '59', '60', '69', '70', '81', '57', '61', '72', '11', '13', '14', '15', '52', '54', '55', '33', '56', '21', '27', '62', '63', '65', '68', '82', '43', '46', '75', '47', '74', '67'))

In [67]:
graphs[1].nodes()

NodeView(('1', '32', '2', '3', '36', '5', '6', '39', '8', '7', '77', '34', '4', '37', '28', '48', '42', '12', '45', '80', '49', '53', '38', '10', '33', '9', '11', '16', '17', '18', '35', '41', '83', '13', '14', '20', '30', '31', '51', '52', '54', '57', '59', '76', '79', '19', '23', '21', '22', '24', '61', '62', '63', '55', '15', '56', '40', '25', '26', '29', '78', '27', '65', '75', '69', '73', '81', '82', '43', '46', '64', '66', '68', '70', '71', '44', '47', '74', '50', '58', '60', '72', '67'))

check the below list comprehension, because I'm not 100% confident it's right

In [68]:
graphs = [nx.to_numpy_matrix(g, nodelist=sorted(list(graphs[i].nodes())))
          for i, g in enumerate(graphs)]

### remove diagonals

In [69]:
graphs[0]

matrix([[1.62190e+04, 3.00000e+01, 0.00000e+00, ..., 0.00000e+00,
         9.00000e+00, 0.00000e+00],
        [3.00000e+01, 3.90200e+04, 6.11000e+02, ..., 0.00000e+00,
         7.36300e+03, 6.54900e+03],
        [0.00000e+00, 6.11000e+02, 9.15300e+03, ..., 0.00000e+00,
         0.00000e+00, 0.00000e+00],
        ...,
        [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 2.93400e+03,
         0.00000e+00, 0.00000e+00],
        [9.00000e+00, 7.36300e+03, 0.00000e+00, ..., 0.00000e+00,
         1.03801e+05, 2.12200e+03],
        [0.00000e+00, 6.54900e+03, 0.00000e+00, ..., 0.00000e+00,
         2.12200e+03, 1.83070e+04]])

In [70]:
for i, _ in enumerate(graphs):
    np.fill_diagonal(graphs[i], 0)

### consider LLC somehow

In [ ]:
# ???

### make single output graph

In [ ]:
# for each element of graphs, flatten that element
# then, append that element as a row in the final output matrix

### save as csv

In [ ]:
# np.savetxt('HBN_output.csv', graph)